In [3]:
%%capture
!pip install transformers
! pip install git+https://github.com/openai/CLIP.git

In [2]:
#@title Drive Downloader

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

download_with_pydrive = True #@param {type:"boolean"}

class Downloader(object):
    def __init__(self, use_pydrive):
        self.use_pydrive = use_pydrive

        if self.use_pydrive:
            self.authenticate()

    def authenticate(self):
        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        self.drive = GoogleDrive(gauth)

    def download_file(self, file_id, file_dst):
        if self.use_pydrive:
            downloaded = self.drive.CreateFile({'id':file_id})
            downloaded.FetchMetadata(fetch_all=True)
            downloaded.GetContentFile(file_dst)
        else:
            !gdown --id $file_id -O $file_dst

downloader = Downloader(download_with_pydrive)

In [4]:
#@title Imports

import clip
import os
from torch import nn
import numpy as np
import torch
import torch.nn.functional as nnf
import sys
from typing import Tuple, List, Union, Optional
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
from google.colab import files
import skimage.io as io
import PIL.Image
from IPython.display import Image


N = type(None)
V = np.array
ARRAY = np.ndarray
ARRAYS = Union[Tuple[ARRAY, ...], List[ARRAY]]
VS = Union[Tuple[V, ...], List[V]]
VN = Union[V, N]
VNS = Union[VS, N]
T = torch.Tensor
TS = Union[Tuple[T, ...], List[T]]
TN = Optional[T]
TNS = Union[Tuple[TN, ...], List[TN]]
TSN = Optional[TS]
TA = Union[T, ARRAY]


D = torch.device
CPU = torch.device('cpu')


def get_device(device_id: int) -> D:
    if not torch.cuda.is_available():
        return CPU
    device_id = min(torch.cuda.device_count() - 1, device_id)
    return torch.device(f'cuda:{device_id}')


CUDA = get_device

current_directory = os.getcwd()
save_path = os.path.join(os.path.dirname(current_directory),'working', "pretrained_models")
os.makedirs(save_path, exist_ok=True)

In [6]:
import os

uploaded_path = "/kaggle/input/model_weight0/pytorch/default/1/"
print(os.listdir(uploaded_path))


['Copy of 0.pt']


In [7]:
import shutil

source_path = "/kaggle/input/model_weight0/pytorch/default/1/Copy of 0.pt"  # Use the correct filename
destination_folder = "/kaggle/working/pretrained_models"

# Ensure destination folder exists
os.makedirs(destination_folder, exist_ok=True)

destination_path = os.path.join(destination_folder, os.path.basename(source_path))

shutil.copy(source_path, destination_path)

print(f"Model moved successfully to {destination_path}!")


Model moved successfully to /kaggle/working/pretrained_models/Copy of 0.pt!


In [8]:
model_path = os.path.join(save_path, 'Copy of 0.pt')

# Model

In [9]:
#@title Model

class ClipCaptionModel(nn.Module):

    def get_dummy_token(self, batch_size: int, device: torch.device) -> torch.Tensor:
        return torch.zeros(batch_size, self.prefix_length, dtype=torch.int64, device=device)

    def forward(self, tokens: torch.Tensor, prefix: torch.Tensor, mask: Optional[torch.Tensor] = None,
                labels: Optional[torch.Tensor] = None):
        embedding_text = self.gpt.transformer.wte(tokens)
        prefix_projections = self.clip_project(prefix).view(-1, self.prefix_length, self.gpt_embedding_size)
        embedding_cat = torch.cat((prefix_projections, embedding_text), dim=1)
        if labels is not None:
            dummy_token = self.get_dummy_token(tokens.shape[0], tokens.device)
            labels = torch.cat((dummy_token, tokens), dim=1)
        out = self.gpt(inputs_embeds=embedding_cat, labels=labels, attention_mask=mask)
        return out

    def __init__(self):
        super(ClipCaptionModel, self).__init__()
        self.prefix_length = 40
        self.gpt = GPT2LMHeadModel.from_pretrained('gpt2')
        self.gpt_embedding_size = self.gpt.transformer.wte.weight.shape[1]
        self.clip_project = TransformerMapper(640, self.gpt_embedding_size, 40,
                                                                     40, 8)



class MLP(nn.Module):

    def forward(self, x: T) -> T:
        return self.model(x)

    def __init__(self, sizes: Tuple[int, ...], bias=True, act=nn.Tanh):
        super(MLP, self).__init__()
        layers = []
        for i in range(len(sizes) -1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=bias))
            if i < len(sizes) - 2:
                layers.append(act())
        self.model = nn.Sequential(*layers)



class ClipCaptionPrefix(ClipCaptionModel):

    def parameters(self, recurse: bool = True):
        return self.clip_project.parameters()

    def train(self, mode: bool = True):
        super(ClipCaptionPrefix, self).train(mode)
        self.gpt.eval()
        return self

# Caption prediction

In [56]:
#@title Caption prediction

def generate_beam(model, tokenizer, beam_size: int = 2, prompt=None, embed=None,
                  entry_length=10, temperature=.2, stop_token: str = '.'):

    model.eval()
    stop_token_index = tokenizer.encode(stop_token)[0]
    tokens = None
    scores = None
    device = next(model.parameters()).device
    seq_lengths = torch.ones(beam_size, device=device)
    is_stopped = torch.zeros(beam_size, device=device, dtype=torch.bool)
    with torch.no_grad():
        if embed is not None:
            generated = embed
        else:
            if tokens is None:
                tokens = torch.tensor(tokenizer.encode(prompt))
                tokens = tokens.unsqueeze(0).to(device)
                generated = model.gpt.transformer.wte(tokens)
        for i in range(entry_length):
            outputs = model.gpt(inputs_embeds=generated)
            logits = outputs.logits
            logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)
            logits = logits.softmax(-1).log()
            if scores is None:
                scores, next_tokens = logits.topk(beam_size, -1)
                generated = generated.expand(beam_size, *generated.shape[1:])
                next_tokens, scores = next_tokens.permute(1, 0), scores.squeeze(0)
                if tokens is None:
                    tokens = next_tokens
                else:
                    tokens = tokens.expand(beam_size, *tokens.shape[1:])
                    tokens = torch.cat((tokens, next_tokens), dim=1)
            else:
                logits[is_stopped] = -float(np.inf)
                logits[is_stopped, 0] = 0
                scores_sum = scores[:, None] + logits
                seq_lengths[~is_stopped] += 1
                scores_sum_average = scores_sum / seq_lengths[:, None]
                scores_sum_average, next_tokens = scores_sum_average.view(-1).topk(beam_size, -1)
                next_tokens_source = next_tokens // scores_sum.shape[1]
                seq_lengths = seq_lengths[next_tokens_source]
                next_tokens = next_tokens % scores_sum.shape[1]
                next_tokens = next_tokens.unsqueeze(1)
                tokens = tokens[next_tokens_source]
                tokens = torch.cat((tokens, next_tokens), dim=1)
                generated = generated[next_tokens_source]
                scores = scores_sum_average * seq_lengths
                is_stopped = is_stopped[next_tokens_source]
            next_token_embed = model.gpt.transformer.wte(next_tokens.squeeze()).view(generated.shape[0], 1, -1)
            generated = torch.cat((generated, next_token_embed), dim=1)
            is_stopped = is_stopped + next_tokens.eq(stop_token_index).squeeze()
            if is_stopped.all():
                break
    scores = scores / seq_lengths
    output_list = tokens.cpu().numpy()
    output_texts = [tokenizer.decode(output[:int(length)]) for output, length in zip(output_list, seq_lengths)]
    order = scores.argsort(descending=True)
    output_texts = [output_texts[i] for i in order]
    return output_texts


def generate2(
        model,
        tokenizer,
        tokens=None,
        prompt=None,
        embed=None,
        entry_count=1,
        entry_length=67,  # maximum number of words
        top_p=0.8,
        temperature=1.,
        stop_token: str = '.',
):
    model.eval()
    generated_num = 0
    generated_list = []
    stop_token_index = tokenizer.encode(stop_token)[0]
    filter_value = -float("Inf")
    device = next(model.parameters()).device

    with torch.no_grad():

        for entry_idx in trange(entry_count):
            if embed is not None:
                generated = embed
            else:
                if tokens is None:
                    tokens = torch.tensor(tokenizer.encode(prompt))
                    tokens = tokens.unsqueeze(0).to(device)

                generated = model.gpt.transformer.wte(tokens)

            for i in range(entry_length):

                outputs = model.gpt(inputs_embeds=generated)
                logits = outputs.logits
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)
                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(nnf.softmax(sorted_logits, dim=-1), dim=-1)
                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                                                    ..., :-1
                                                    ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value
                next_token = torch.argmax(logits, -1).unsqueeze(0)
                next_token_embed = model.gpt.transformer.wte(next_token)
                if tokens is None:
                    tokens = next_token
                else:
                    tokens = torch.cat((tokens, next_token), dim=1)
                generated = torch.cat((generated, next_token_embed), dim=1)
                if stop_token_index == next_token.item():
                    break

            output_list = list(tokens.squeeze().cpu().numpy())
            output_text = tokenizer.decode(output_list)
            generated_list.append(output_text)

    return generated_list[0]

In [11]:
#@title GPU/CPU


is_gpu = True #@param {type:"boolean"}


# CLIP model + GPT2 tokenizer

In [14]:
#@title CLIP model + GPT2 tokenizer

device = CUDA(0) if is_gpu else "cpu"
clip_model, preprocess = clip.load("RN50x4", device=device, jit=False)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Mapper

In [12]:
#@title Mapper

class MlpTransformer(nn.Module):
    def __init__(self, in_dim, h_dim, out_d: Optional[int] = None, act=nnf.relu, dropout=0.):
        super().__init__()
        out_d = out_d if out_d is not None else in_dim
        self.fc1 = nn.Linear(in_dim, h_dim)
        self.act = act
        self.fc2 = nn.Linear(h_dim, out_d)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x

class MultiHeadAttention(nn.Module):

    def __init__(self, dim_self, dim_ref, num_heads, bias=True, dropout=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim_self // num_heads
        self.scale = head_dim ** -0.5
        self.to_queries = nn.Linear(dim_self, dim_self, bias=bias)
        self.to_keys_values = nn.Linear(dim_ref, dim_self * 2, bias=bias)
        self.project = nn.Linear(dim_self, dim_self)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, y=None, mask=None):
        y = y if y is not None else x
        b, n, c = x.shape
        _, m, d = y.shape
        # b n h dh
        queries = self.to_queries(x).reshape(b, n, self.num_heads, c // self.num_heads)
        # b m 2 h dh
        keys_values = self.to_keys_values(y).reshape(b, m, 2, self.num_heads, c // self.num_heads)
        keys, values = keys_values[:, :, 0], keys_values[:, :, 1]
        attention = torch.einsum('bnhd,bmhd->bnmh', queries, keys) * self.scale
        if mask is not None:
            if mask.dim() == 2:
                mask = mask.unsqueeze(1)
            attention = attention.masked_fill(mask.unsqueeze(3), float("-inf"))
        attention = attention.softmax(dim=2)
        out = torch.einsum('bnmh,bmhd->bnhd', attention, values).reshape(b, n, c)
        out = self.project(out)
        return out, attention


class TransformerLayer(nn.Module):

    def forward_with_attention(self, x, y=None, mask=None):
        x_, attention = self.attn(self.norm1(x), y, mask)
        x = x + x_
        x = x + self.mlp(self.norm2(x))
        return x, attention

    def forward(self, x, y=None, mask=None):
        x = x + self.attn(self.norm1(x), y, mask)[0]
        x = x + self.mlp(self.norm2(x))
        return x

    def __init__(self, dim_self, dim_ref, num_heads, mlp_ratio=4., bias=False, dropout=0., act=nnf.relu,
                 norm_layer: nn.Module = nn.LayerNorm):
        super().__init__()
        self.norm1 = norm_layer(dim_self)
        self.attn = MultiHeadAttention(dim_self, dim_ref, num_heads, bias=bias, dropout=dropout)
        self.norm2 = norm_layer(dim_self)
        self.mlp = MlpTransformer(dim_self, int(dim_self * mlp_ratio), act=act, dropout=dropout)


class Transformer(nn.Module):

    def forward_with_attention(self, x, y=None, mask=None):
        attentions = []
        for layer in self.layers:
            x, att = layer.forward_with_attention(x, y, mask)
            attentions.append(att)
        return x, attentions

    def forward(self, x, y=None, mask=None):
        for i, layer in enumerate(self.layers):
            if i % 2 == 0 and self.enc_dec: # cross
                x = layer(x, y)
            elif self.enc_dec:  # self
                x = layer(x, x, mask)
            else:  # self or cross
                x = layer(x, y, mask)
        return x

    def __init__(self, dim_self: int, num_heads: int, num_layers: int, dim_ref: Optional[int] = None,
                 mlp_ratio: float = 2., act=nnf.relu, norm_layer: nn.Module = nn.LayerNorm, enc_dec: bool = False):
        super(Transformer, self).__init__()
        dim_ref = dim_ref if dim_ref is not None else dim_self
        self.enc_dec = enc_dec
        if enc_dec:
            num_layers = num_layers * 2
        layers = []
        for i in range(num_layers):
            if i % 2 == 0 and enc_dec:  # cross
                layers.append(TransformerLayer(dim_self, dim_ref, num_heads, mlp_ratio, act=act, norm_layer=norm_layer))
            elif enc_dec:  # self
                layers.append(TransformerLayer(dim_self, dim_self, num_heads, mlp_ratio, act=act, norm_layer=norm_layer))
            else:  # self or cross
                layers.append(TransformerLayer(dim_self, dim_ref, num_heads, mlp_ratio, act=act, norm_layer=norm_layer))
        self.layers = nn.ModuleList(layers)


class TransformerMapper(nn.Module):

    def forward(self, x):
        x = self.linear(x).view(x.shape[0], self.clip_length, -1)
        prefix = self.prefix_const.unsqueeze(0).expand(x.shape[0], *self.prefix_const.shape)
        prefix = torch.cat((x, prefix), dim=1)
        out = self.transformer(prefix)[:, self.clip_length:]
        return out

    def __init__(self, dim_clip: int, dim_embedding: int, prefix_length: int, clip_length: int, num_layers: int = 8):
        super(TransformerMapper, self).__init__()
        self.clip_length = clip_length
        self.transformer = Transformer(dim_embedding, 8, num_layers)
        self.linear = nn.Linear(dim_clip, clip_length * dim_embedding)
        self.prefix_const = nn.Parameter(torch.randn(prefix_length, dim_embedding), requires_grad=True)

class ClipCaptionModel(nn.Module):

    def get_dummy_token(self, batch_size: int, device: torch.device) -> torch.Tensor:
        return torch.zeros(batch_size, self.prefix_length, dtype=torch.int64, device=device)

    def forward(self, tokens: torch.Tensor, prefix: torch.Tensor, mask: Optional[torch.Tensor] = None,
                labels: Optional[torch.Tensor] = None):
        embedding_text = self.gpt.transformer.wte(tokens)
        prefix_projections = self.clip_project(prefix).view(-1, self.prefix_length, self.gpt_embedding_size)
        embedding_cat = torch.cat((prefix_projections, embedding_text), dim=1)
        if labels is not None:
            dummy_token = self.get_dummy_token(tokens.shape[0], tokens.device)
            labels = torch.cat((dummy_token, tokens), dim=1)
        out = self.gpt(inputs_embeds=embedding_cat, labels=labels, attention_mask=mask)
        return out

    def __init__(self, prefix_size: int = 640,
                 num_layers: int = 8):
        super(ClipCaptionModel, self).__init__()
        self.prefix_length = 40
        self.gpt = GPT2LMHeadModel.from_pretrained('gpt2')
        self.gpt_embedding_size = self.gpt.transformer.wte.weight.shape[1]
        self.clip_project = TransformerMapper(prefix_size, self.gpt_embedding_size, 40,
                                                                     40, num_layers)





# Load model weights

In [16]:
#@title Load model weights
model = ClipCaptionModel()


model.load_state_dict(torch.load(model_path, map_location=CPU),strict=False)
model = model.eval()
device = CUDA(0) if is_gpu else "cpu"
model = model.to(device)


<ipython-input-16-612a68ae9a54>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=CPU),strict=False)


# Text Inference

In [ ]:
!ps -aux | grep -E 'torch|tensorflow|cuda|nvidia'


In [ ]:
!ps -eo pid,command | grep python


In [ ]:
!ps -o ppid= -p 102


In [ ]:
!kill -9  1


In [ ]:
import torch

import torch
torch.cuda.empty_cache()
torch.cuda.ipc_collect()


In [21]:
!nvidia-smi


Fri Feb 28 13:06:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P0             33W /  250W |    1979MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!kill -9 3852


In [39]:
ms_coco_labels = [',pimtaom', '-', '1', 'abacus', 'abinet', 'accordion', 'piano accordion', 'squeeze box', 'acropolis', 'ad', 'adding machine', 'totalizer', 'totaliser', 'air conditioner', 'air freshener', 'aircraft', 'aircraft carrier', 'airlock', 'airpl', 'airplane', 'airport cart', 'alarm', 'alarm clock'] 
                  
                  # 'alembic', 'alga', 'altar', 'altarpiece', 'amphitheater, amphitheatre, coliseum', 'amphora', 'amplifier', 'anchor', 'andiron', 'animal', 'animal topiary', 'antenna', 'antler', 'anvil', 'apparel', 'apple', 'apron', 'aquarium', 'aqueduct', 'arc', 'arcade', 'arcade machine', 'arcade machines', 'arch', 'armchair', 'armor, armour', 'art', 'art mannequin', 'artificial golf green', 'asdf', 'ashcan', 'ashtray', 'asymmetric bars', 'autoclave, sterilizer, steriliser', 'autopsy table', 'auxiliary trolley', 'aviary', 'avocados', 'award', 'awards', 'awning', 'ax', 'baby buggy', 'baby chair', 'baby cheetah', 'baby lion', 'baby rhinoceros', 'baby walker', 'baby weighs', 'backdrop', 'backdrops', 'background', 'backp', 'backpack', 'backpacks', 'badge', 'badlands', 'badlands hills', 'bag', 'baggage carts', 'bagpipes', 'bakery items', 'balcony', 'balcony covered', 'ball', 'ball rack', 'ball stands', 'ballon', 'ballons', 'balloon', 'balloons', 'ballots', 'balls rack', 'baloons', 'bamboo', 'bamboo canes', 'banana', 'bananas', 'bandstand', 'banjo', 'banker', 'banner', 'banners', 'bannister', 'baptismal font', 'baptistry', 'bar', 'bar lift', 'barbecue', 'barbed wire', 'barbell', 'barn', 'barrack buildings', 'barrel', 'barrel rack', 'barrels', 'barren land', 'barricade', 'barrier', 'bascket', 'base', 'baseball bat', 'baseball field', 'baseball glove', 'baseboard', 'baseboards', 'basin', 'basket', 'basketball', 'basketball court', 'baskets', 'bass drum', 'bassinet', 'bat', 'bath sponge', 'bathrobe', 'bathroom tiles', 'bathtub', 'batting cage', 'beach lake', 'beach wrap', 'beacon', 'beam', 'beams', 'bear', 'bear cutout', 'bed', 'bed runner', 'bed trolley', 'beds', 'bedspread', 'bee', 'beer machine', 'beer pump', 'bell', 'belt', 'belt loader', 'belt rack', 'belts', 'bench', 'berbiqui', 'berth', 'bib', 'bicycle', 'bicycle rack', 'bicycle racks', 'bidet', 'bidon', 'big top', 'big wheel', 'bilding', 'bildings', 'billboard', 'billiard ball', 'billiard balls', 'bin', 'binder', 'binoculars', 'bird', 'birdcage', 'birds', 'biscuit', 'bitch', 'blackboard', 'blade, vane', 'blanket', 'blankets', 'blast furnace', 'bleachers', 'blender', 'blind', 'blinder', 'blinds', 'block', 'blouse', 'blueberry', 'blusher brush', 'boaard', 'boar', 'board', 'board game', 'boards', 'boardwalk', 'boat', 'boat stop', 'boat trolley', 'body', 'bog water', 'boiler', 'boillers', 'bollard', 'bolt', 'bomb', 'bone', 'bones', 'bonfire', 'bonnet', 'bonsai tree', 'bood', 'book', 'book rack', 'book stand', 'bookcase', 'bookend', 'bookends', 'booklet', 'bookstall', 'boomerang', 'boot', 'booth', 'booths', 'botijo', 'botlle', 'bottle', 'bottle pack', 'bottle rack', 'bough', 'boulder', 'bounce seat', 'bouquet', 'bow', 'bow window', 'bowl', 'bowling alley', 'bowling alleys', 'bowling ball return', 'bowling pins', 'bowls', 'box', 'box jury', 'box office', 'box stands', 'box-file', 'boxing gloves', 'bracelet', 'bracelets', 'bracket', 'brake pedal', 'branch', 'branches', 'bread', 'bread basket', 'bread box', 'bread roll', 'bread rolls', 'bread tray', 'breadbasket', 'breads', 'breaker box', 'breakwater', 'brewery', 'brick', 'bricks', 'bridge', 'bridges', 'briefcase', 'broccoli', 'brochure', 'brochures rack', 'broom', 'brush', 'brushes', 'bucket', 'buckets', 'bud', 'buds', 'buffet', 'buggy', 'buildigs', 'building', 'building arena', 'building materials', 'building ziggurat', 'buildings factory', 'buiulding', 'bulbs', 'bull', 'bull skull', 'bulldozer', 'bulletin board', 'bullfighter', 'bullring', 'bulstrade', 'bumper', 'bunch', 'bunch grapes', 'bunk', 'bunk bed', 'buns', 'buoy', 'buoys', 'burner', 'bus', 'bus station', 'bus stop', 'buses', 'business card', 'business cards', 'butane gas cylinder', 'butchery', 'butter', 'butterfly', 'button box', 'buttress', 'buzzer', 'bycicle carriage', 'caabinet', 'caar', 'cabbages', 'cabin', 'cabine', 'cabinet', 'cabins', 'cable', 'cable car', 'cable railway', 'cables', 'cabomet', 'cabubet', 'caca', 'cactus', 'caffee table', 'cage', 'cages', 'cake', 'calculator', 'calendar', 'call buttons', 'camcorder', 'camel', 'camels', 'camera', 'camouflage', 'camping stove', 'can', 'can opener', 'canal', 'canal water', 'candelabrum', 'candies', 'candle', 'candles', 'candlestick', 'candy', 'candy bag', 'candy bags', 'candy track', 'cane', 'canes', 'canister', 'canlendar', 'cannon', 'canoe', 'canopy', 'canvas', 'canvases', 'canyon', 'canyons', 'cap', 'cape', 'caps', 'car', 'car scrapping', 'car wash', 'car wreck', 'carapace', 'caravan', 'caravans', 'card', 'card display', 'card game', 'card table', 'cardboard', 'cargo', 'carousel', 'carousel horse', 'carp', 'carper', 'carpets', 'carport', 'carriage', 'carrot', 'cars scraping', 'cars scrapping', 'cart', 'carts', 'case', 'cases', 'cash machine', 'cash register', 'casing interior', 'casket', 'casse', 'casserole', 'cassette', 'cassettes', 'castle', 'castle in ruins', 'cat', 'catamaran', 'catch', 'caterpillar', 'catwalk', 'cauldron', 'cave', 'cave entrance', 'cavern', 'cceiling', 'cd', 'cd box', 'cd player', 'cd rack', 'cd shelf', 'cd stack', 'cd stand', 'cds', 'cds box', 'ceiling', 'ceiling controls', 'ceiling decor', 'ceiling lamp', 'ceiling piece', 'cell', 'cell door', 'cell phone', 'cello', 'cello cover', 'cells', 'cellular telephone', 'cement', 'centerpiece', 'central processing unit', 'central reservation', 'centrifuge', 'ceramic box', 'ceramic hob', 'ceramic pot', 'ceramic ware', 'ceramics', 'ceramics box', 'certificate', 'ceukubg', 'chain', 'chains', 'chainsaw', 'chair', 'chair desk', 'chair lift', 'chairr', 'chairs desk', 'chairs lift', 'chais', 'chaise lounge', 'chalk', 'chandelier', 'change machine', 'channel', 'channel water', 'check-in-desk', 'check-in-desks', 'checkers table', 'checkpoint', 'cheese', 'ches table', 'chess', 'chess table', 'chest', 'chest of drawers', 'chick', 'chicken', 'chicks', 'childish games', 'chimney', 'chimneys', 'chinese lantern', 'chip', 'chippings', 'chips', 'chocolate', 'chocolate cakes', 'chocolates', 'chopping board', 'chorizo', 'chorizos', 'chothing', 'christmas lights', 'christmas tree', 'church', 'chute', 'cigar box', 'cigarette packet', 'ciment mixer', 'cinch', 'cistern', 'city', 'clabe car', 'clamp lamp', 'clamp spotlight', 'clams', 'clarinet', 'clay', 'cliff', 'cliff and waterfall', 'climber person', 'climbing frame', 'clipboard', 'cloak', 'clock', 'clock radio', 'clocks', 'closet', 'closet rod', 'cloth material', 'clothes hanging', 'clothes horse', 'clothes stand', 'clothesline', 'clothing', 'clothing rack', 'cloud', 'cloudy sky', 'club', 'clubs', 'clutch pedal', 'coast', 'coaster', 'coasters', 'coastline', 'coat', 'coat of arms', 'coat rack', 'coat stand', 'coats', 'cobbles', 'cock', 'cockpit', 'cockpot', 'cocktail shaker', 'coconut', 'cocoon', 'coffee', 'coffee cup', 'coffee maker', 'coffee pot', 'coffee table', 'coffin', 'colander', 'cold storage', 'cold storage door', 'cold store', 'colour board', 'colum', 'columbarium', 'columbarium niche', 'column', 'comb', 'combine harvester', 'comforter', 'comic strip', 'compact discs', 'compartment', 'compas', 'compueter', 'computer', 'computer console', 'concrete mixer', 'cone', 'confessional booth', 'console', 'console table', 'construction equipment', 'contact lens case', 'container', 'containers', 'contrabass', 'control cabinet', 'control panel', 'control panel, instrument panel, control board, board, panel', 'control panels', 'control room', 'control table', 'control tower', 'controls', 'conveyer belt', 'cooker', 'cookie', 'cookies', 'cooks', 'cool box', 'cooler', 'cooling tower', 'coop', 'copier', 'copper vessel', 'coral', 'cords', 'cork stopper', 'corkscrew', 'cornice', 'cotton', 'couch', 'counter', 'counters', 'countertop', 'counterweight', 'court', 'courts', 'cove', 'cover', 'cover curtain', 'covered bridge', 'covered wagon', 'cow', 'cows', 'cradle', 'crafts', 'crane', 'crane scrapping', 'crane truck', 'cranes', 'crank', 'craps table', 'crate', 'crater', 'crates', 'cream', 'cream tube', 'creamer', 'creek', 'creeper', 'crevasse', 'crib', 'crock', 'crock pot', 'crock rack', 'crockery', 'cross', 'crosses', 'crossing, crosswalk, crossover', 'cruet', 'cruet set', 'crumb', 'crutch', 'cub', 'cube', 'cubicles', 'cue', 'cue place', 'cue rack', 'cues', 'cumputer', 'cup', 'cups', 'curb', 'curb cut', 'curbstone', 'curtain', 'curtain cover', 'curtains', 'cushion', 'cushions', 'cut slices machine', 'cut table', 'cutlery', 'cutter', 'cyclotron', 'cylinder', 'cymbal', 'dam', 'dartboard', 'dashboard', 'debris', 'decanter', 'deck', 'deck boat', 'deck chair', 'deck chairs', 'deck house', 'decoder', 'decor', 'decoration', 'decorations', 'decorative glass', 'decorative wood', 'deer', 'deer head', 'degree certificate', 'demolition house', 'dental chair', 'dental instruments', 'dental sink', 'dental swivel chair', 'deodorant', 'desert', 'desert field', 'desert ground', 'desk', 'desk chair', 'desk mat', 'desk organizer', 'developing machine', 'dial', 'dice', 'digger', 'digital clock', 'dining table', 'diploma', 'dipper', 'directory', 'dirt track', 'dish', 'dish drainer', 'dish rack', 'dish soap', 'dish towel', 'dishes', 'dishrack', 'dishrag', 'dishwasher', 'disk', 'disk spindle', 'disks', 'dispenser', 'display board', 'display window', 'display, video display', 'disposal plant', 'distaff', 'ditch', 'diver', 'diving board', 'diving goggles', 'dlooe', 'doble door', 'docks', 'document', 'dog', 'dog bed', 'dog dish', 'doily', 'doll', 'dollhouse', 'dolmen', 'dolmens', 'dolphin', 'dome', 'donkey', 'donut', 'doo', 'door', 'door bars', 'doorbell', 'doorframe', 'doort', 'doorway arch', 'dormer', 'double door', 'dppr', 'drain', 'drain pipe', 'drain plug', 'drain vent', 'drapes', 'drawer', 'drawing', 'drawing board', 'drawing table', 'drawing tables', 'drawings', 'dressing screen', 'dressing table', 'dried flowers', 'dried herbs', 'drift ice', 'drill', 'drill bits', 'drill rig', 'drilling equipment', 'drinking fountain', 'driveway', 'drom', 'drum', 'drums', 'dry dock', 'dryer', 'duck', 'duck shelter', 'ducks', 'dumbbell', 'dumbbells', 'dummy', 'dune', 'dunes', 'dust', 'duster', 'dustpan', 'dvd', 'dvd box', 'dvd boxes', 'dvd player', 'dvd players', 'dvds', 'dvds box', 'dvds rack', 'dye', 'ear', 'earphones', 'earring', 'earrings', 'ears', 'earth', 'earthenware jar', 'easel', 'eaves', 'egg', 'egg cup', 'egg tray', 'eggs', 'eiderdown', 'electric cable', 'electric toothbrush', 'electrical plant', 'electrical substation', 'electricity box', 'electricity meter', 'electronic devise', 'electronic organ', 'elephant', 'elevator', 'elevator door', 'elevator doors', 'elliptical machine', 'embankment', 'ember', 'embrasure', 'engine', 'entrance', 'entrance parking', 'entry', 'entry phone', 'envelope', 'envelopes', 'equalizer', 'equestrian obstacle', 'equipment', 'erase', 'escalator', 'escarpment', 'estuary', 'excavator', 'exercise bench', 'exercise bike', 'exercise bikes', 'exercise machines', 'exhi', 'exhibitor', 'exit sign', 'expedition', 'extension cord', 'external driver', 'external drivers', 'extractor fan', 'extractor pipe', 'eyeglasses', 'fabric', 'fabric softener', 'fabrics', 'face, human face', 'factory', 'fags', 'fairground', 'fairground attraction', 'fairground ride', 'fan', 'faucet', 'fax', 'fax machine', 'feather', 'feed', 'feeder', 'feeding bottle', 'feeding trough', 'fence', 'fence rubble', 'fences', 'fende', 'fender', 'fern', 'fernce', 'ferris wheel', 'field', 'figure', 'figurine', 'file', 'file box', 'file boxes', 'file cabinet', 'file cabinets', 'file organizer', 'files', 'filing cabinets', 'finger', 'fire', 'fire alarm', 'fire alarm box', 'fire blanket', 'fire booth', 'fire engine', 'fire escape', 'fire extinguisher', 'fire hose', 'fire hydrant', 'fire truck', 'fire utensils', 'fireplaace', 'fireplace', 'fireplace utensils', 'fireplug', 'firewood', 'fireworks', 'first floor', 'fish', 'fish farm water', 'fish structure', 'fish tank', 'fishing pole', 'fishing rod', 'fitting room', 'flag', 'flag pole', 'flags', 'flagstones', 'flame', 'flashlight', 'flask', 'flight deck', 'flip flop', 'flip flops', 'float', 'floating bridge', 'flock of sheep', 'flood', 'floodlight', 'floor', 'floor fan', 'floor light', 'floor recessed light', 'floor spotlight', 'floot', 'flower', 'flower basket', 'flowerpots', 'flowrs', 'fluoresc', 'fluorescent', 'fluorescet tube', 'flusher', 'flute', 'flyer', 'flyers', 'flyscreen', 'foal', 'foam', 'fog', 'fog bank', 'folded box', 'folders', 'folding chair', 'folding chairs', 'folding door', 'folding doors', 'folding screen', 'folding table', 'foliage', 'font', 'food', 'food cart', 'food processor', 'foodstuffs', 'foot rest', 'football', 'football stadium', 'football table', 'footbridge', 'footpath', 'footrest', 'forecourt', 'forest', 'forge', 'fork', 'forklift', 'fort', 'fountain', 'frame', 'frames', 'freezer', 'freezing ground', 'freight goods', 'freight train', 'frisbee', 'front, front end, forepart', 'fruit', 'fruit bowl', 'fruit machine', 'fruit stand', 'fryer', 'fryer basket', 'frying pan', 'frying pans', 'fuel can', 'fuel pump', 'fuel station', 'fuel tank', 'fumes', 'funfair', 'funicular', 'funnel', 'fur', 'furnace', 'furnace room', 'furrow', 'game', 'game table', 'games', 'games room', 'games table', 'gaming table', 'gangway', 'gap', 'garage', 'garbage', 'garland', 'garlic', 'garlics', 'garment bag', 'gas bottle', 'gas cap', 'gas cylinder', 'gas heater', 'gas meter', 'gas pedal', 'gas pump', 'gas station', 'gas well', 'gasworks', 'gate', 'gatehouse', 'gates', 'gateway', 'gauge', 'gazebo', 'gear', 'gear shift', 'gear wheel', 'gears', 'gearshift', 'generator', 'giraffe', 'girder', 'glacier', 'glass', 'glass box', 'glass separation', 'glass wall', 'gliue', 'glove', 'glove compartment', 'gloves', 'glue', 'goal', 'goat', 'goblet', 'goggles', 'goldfish bowl', 'golf bag', 'golf ball', 'golf cart', 'golf club', 'golf green', 'golf sticks', 'golfer', 'golg club', 'gondola', 'goods', 'goose', 'gooses', 'gradstand', 'graiting', 'gramophone', 'grand piano', 'grandfather clock', 'grandstand', 'grapes', 'grapevine', 'graphic equalizer', 'grass', 'grass field', 'grassland', 'grating', 'graund', 'gravel', 'gravestone', 'gravestones', 'gravy boat', 'greehouse', 'greenhouse', 'grid', 'griddle', 'grill', 'grille door', 'grills', 'grinder', 'gripper', 'grotto', 'ground desert', 'ground leaves', 'ground sand', 'ground shrubs', 'groundd', 'grpimd', 'guard rail', 'guardhouse', 'guillotine', 'guitar', 'guitar case', 'guitar cover', 'gull', 'gun', 'gutter', 'gym machine', 'gym shoe', 'hair', 'hair drier', 'hair dryer', 'hair spray', 'hairbrush', 'hairdresser chair', 'hairdressers chair', 'hairpieces', 'half globe', 'hall', 'ham', 'hammer', 'hammock', 'hams', 'hand bellows', 'hand dryer', 'hand trolley', 'handbag', 'handle', 'handr', 'handrair', 'hands', 'hangar', 'hanger', 'hanger hook', 'hangers', 'hanging clothes', 'hanging instrument', 'hanging utensils', 'harbor', 'hard drive', 'hard shoulder', 'hardware', 'harness', 'harp', 'harpoon', 'hat', 'hatch', 'hatchery', 'hats', 'haversack', 'hay', 'hay bale', 'hay stacks', 'head deer', 'head sport', 'head, caput', 'headboard', 'headlight, headlamp', 'headphones', 'headstone cross', 'headstones', 'heart', 'hearth', 'heater', 'heating', 'hedge', 'hedge partial', 'helic', 'helicopter', 'heliport', 'helmet', 'hen', 'hen house', 'henhouse', 'hens', 'herb', 'highlight', 'highway bridge', 'hill', 'hillside', 'hinge', 'hippopotamus head', 'hjouse', 'hoarding', 'hobby-elephant', 'hobby-horse', 'hockey rink', 'hockey table', 'hoe', 'hoist', 'hole', 'hole puncher', 'hollow', 'hood', 'hook', 'horizontal bar', 'horse', 'horse carousel', 'hose', 'hosepipe', 'hot air balloon', 'hot dog', 'hot mitt', 'hot tub', 'hotel safe', 'hotlp', 'hotplate', 'hourglass', 'house', 'house number', 'houseboat', 'housee', 'housing project', 'hovel', 'howdah', 'hull', 'hurdle', 'hutch', 'huts', 'ice', 'ice bucket', 'ice cone', 'ice cream', 'ice cream machine', 'ice creme stand', 'ice decoration', 'ice field', 'ice floe', 'ice ground', 'ice hockey rink', 'ice mountain', 'ice rink', 'ice shelf', 'ice track', 'ice wall', 'iceberg', 'icicle', 'icicles', 'id', 'id card', 'idol', 'igloo', 'in box', 'incense', 'incinerator', 'incubator', 'index card', 'index cards', 'indoor track', 'industrial building', 'industrial machine', 'industrial plant', 'industrial robot', 'industry', 'inflatable bounce game', 'inflatable doll', 'inflatable glove', 'inflatable park', 'inflatable toy', 'inflatable train', 'ingot', 'instrument', 'instrument control', 'instrument panel', 'ipod', 'iron', 'iron bar', 'iron cross', 'iron po', 'iron pot', 'iron tubes', 'ironing board', 'island', 'islet', 'iv bag', 'jack', 'jacket', 'japanese lantern', 'jar', 'jellyfish', 'jersey', 'jetty', 'jewelry display', 'jground', 'joist', 'joists', 'joystick', 'judge table', 'jug', 'juice machine', 'juke box', 'junk', 'junk pile', 'kasbah', 'kayak', 'kennel', 'kennels', 'kettle', 'key', 'key pad', 'keyboaard', 'keyboard', 'killer whale', 'kit', 'kitchen', 'kitchen island', 'kitchen towel', 'kitchen utensils', 'kite', 'kite string', 'knife', 'knob', 'knobs', 'lab bench', 'label', 'label tag', 'laboratory equipment', 'laboratory machine', 'laboratory machines', 'labyrinth', 'ladder', 'ladle', 'lagoon', 'lagoon water', 'lake', 'lake house', 'laminating machine', 'lamp', 'lamp cord', 'lamp post', 'lamps', 'lampshade', 'lance', 'land', 'landfill', 'landing track', 'landing tracks', 'lantern', 'lanterns', 'laptop', 'laptop bag', 'large fork', 'lathe', 'lattice', 'launch pad', 'laundry', 'laundry basket', 'lava', 'lawn mower', 'leaf', 'leash', 'leather', 'leathers', 'leaves', 'lectern', 'ledge', 'ledges', 'leg', 'lemon', 'lemons', 'letter mail', 'lettuce', 'level', 'lever', 'lgiht post', 'license plate, numberplate', 'lid', 'life preserver', 'lift', 'light', 'light aircraft', 'light bulb', 'light display', 'light house', 'light pole', 'light sw', 'light switches', 'light tower', 'lighter', 'lighting', 'lightning', 'linens', 'lion', 'lithograph', 'litter basket', 'livestock', 'loaves', 'lobster', 'lock', 'locker', 'lockers', 'lockgate', 'lodge', 'loft', 'log', 'logo', 'logs', 'loom', 'loudspeaker', 'lower deck', 'luggage', 'luggage rack', 'lumber', 'lumps', 'mace', 'machine', 'machine gun', 'machinery', 'machines', 'magazine', 'magazine rack', 'magnet', 'magnifying glass', 'make', 'makeup', 'mallet', 'manhole', 'mannequin', 'manometer', 'mantle', 'mantle piece', 'manufacturing plant', 'map', 'maps', 'maracas', 'margarine', 'marker', 'market', 'marking pens', 'marsh water', 'mask', 'massage bed', 'massage table', 'mast', 'mat', 'matchbox', 'material', 'material rolls', 'materials', 'mattress', 'mausoleum', 'maze', 'measurer', 'meat', 'meat bag', 'meat bags', 'meat hooks', 'meat piece', 'meat pieces', 'mechanical shovel', 'mechanism', 'medal', 'medical machine', 'medicines', 'megaphone', 'menu', 'menu card', 'menu display', 'menus', 'merchandise', 'merry-go-round', 'mesh guard rail', 'message board', 'metal decoration', 'metal piping', 'metal profiles', 'metal shutters', 'metal structure', 'meter', 'meter box', 'meters', 'mezzanine', 'microbrewery', 'microph', 'microphone', 'microscope', 'microw', 'microwave', 'milestone', 'mill', 'millrace', 'minar', 'minaret', 'minced meat', 'mine', 'mineral', 'miniature chair', 'miniature sofa', 'minibike', 'mirror', 'mirrors', 'missile', 'mitra', 'mitten', 'mixer', 'mixing board', 'mobile', 'mobile conveyor', 'mobile home', 'model', 'model dinosaur', 'modern art', 'mold', 'molding', 'money', 'monitor', 'monitors', 'monkey', 'monolith', 'monument', 'moon', 'moon decoration', 'mooring', 'moose', 'mop handle', 'mortar', 'mosaic', 'mosque', 'moss', 'motor', 'motorbike cart', 'motorboat', 'motorboats', 'motorcycle', 'moulding', 'mound', 'mountain', 'mountain ledge', 'mountain pass', 'mountain salt', 'mountain terraces', 'mouse', 'mouse stand', 'mousepad', 'movie poster', 'moving walkway', 'muffin', 'mug', 'mug set', 'multiple plug', 'multiple socket', 'mural', 'mushroom', 'music box', 'music control table', 'music devices', 'music machine', 'music stand', 'musical instruments', 'musicians place', 'mussels', 'nail brush', 'name', 'name plate', 'names', 'napkin', 'napkin holder', 'napkin rack', 'napkin ring', 'napking', 'necklace', 'necklaces', 'necktie', 'nest', 'net', 'netting', 'newspaper', 'newspaper rack', 'newspaper stand', 'niche', 'night light', 'night ttable', 'ninepin', 'nordic comforter', 'note', 'notebokk', 'notebook', 'notecards', 'notepad', 'notes', 'notice', 'notice board', 'npkin', 'nuclear reactor', 'nukestibe', 'numbers', 'numeral', 'nunchaku', 'nuts', 'oar', 'obelisk', 'observation tower', 'observatory', 'obstacle', 'odometer', 'offering', 'office', 'office partition', 'office window', 'offices', 'oil', 'oil lamp', 'oil tube', 'oil tubes', 'oilcloth', 'oilfield', 'olives', 'onions', 'operation table', 'orange', 'orchard', 'orchestra', 'organ', 'organizer', 'oriental lantern', 'orifice, opening, porta', 'ornament', 'ornamental box', 'ornamental structure', 'ornaments', 'otlet', 'ottoman', 'outhouse', 'outhouse wc', 'oven', 'overhang', 'overpass', 'oxygen tank', 'oyster bank', 'p', 'paath', 'pack', 'pack bags', 'package', 'packages', 'packet', 'packets', 'pad', 'pad of paper', 'paddle', 'paddles', 'padlock', 'padlocks', 'pagoda', 'paht', 'painiting', 'paint box', 'paint brush', 'paint brushes', 'paint bucket', 'paint can', 'paint pot', 'paintbrush', 'painting', 'paintings', 'paints', 'pala', 'palette', 'palisade', 'pallet', 'pallets', 'palm', 'pan', 'pane', 'panel', 'panelling', 'panels', 'panes', 'pans', 'pant', 'pant pot', 'pantheon', 'pantry', 'pants', 'paper', 'paper box', 'paper cups', 'paper dishes', 'paper dispenser', 'paper file', 'paper filer', 'paper napkin', 'paper napkins', 'paper rack', 'paper roll', 'paper rolls', 'paper towel', 'paper towels box', 'paper towels dispenser', 'paper weight', 'paperpile', 'papr', 'parachute', 'park', 'park gear', 'parking', 'parking entrance', 'parking lot', 'parking meter', 'parking place', 'parterre', 'partition', 'partition wall', 'paslm tree', 'pastry', 'path', 'patio', 'patty', 'pavilion', 'pavilions', 'paving stone', 'peak', 'peaked cap', 'peaks', 'pebbles', 'pectoral machine', 'pedestal', 'pelota court', 'pelt', 'pelts', 'pen', 'pen box', 'pen holder', 'pen set', 'pencil', 'pencil case', 'pencil cup', 'pencil sharpener', 'pencils', 'pendant', 'pendant plant', 'penguin', 'pens', 'peoples', 'pepper', 'pepper grinder', 'pepper shaker', 'peppers', 'pergola', 'person', 'person sitting', 'person swimming', 'persons', 'persoon', 'pert', 'pet bowl', 'pet dish', 'petal', 'petrol pump hose', 'petrol pumps', 'petrol station', 'phone charger', 'phone fax', 'phone jack', 'phonends', 'photo', 'photo album', 'photo backdrops', 'photo machine', 'photocopier', 'piano', 'pick', 'pickle', 'picnic spot', 'pictures', 'pie', 'pie spinach', 'piece', 'pieces', 'pier', 'pig', 'pig tail', 'pigeonhole', 'pigsty', 'pile of junk', 'pile of trash', 'pillars', 'pillow', 'pillows', 'pilon', 'pinball', 'pine decor', 'pine decoration', 'pineapple', 'pinecone', 'ping pong paddle', 'pingeonhole', 'pins', 'pipe', 'pipeline', 'pipelines', 'piper', 'pipette', 'piping', 'piprlinrd', 'pit', 'pitch', 'pitcher', 'pitchfork', 'piture', 'pizza', 'placard', 'place card', 'place mat', 'place mats', 'place setting', 'placecard', 'placecard holder', 'planat', 'planats', 'plane', 'plane wreckage', 'planer', 'planetarium', 'planks', 'plans', 'plant', 'plant box', 'plant por', 'plant pots', 'plant stand', 'plant tree', 'plantas', 'planter', 'plants pot', 'plants shrubs', 'plaque', 'plastic', 'plastic bottles', 'plastic cup', 'plastic cups', 'plastic sheet', 'plat', 'plate', 'plate mat', 'plate rack', 'plateau', 'plates rack', 'platf', 'platform', 'platns', 'platre', 'plats', 'platter', 'playground', 'playground equipment', 'playing field', 'plaything', 'pliers', 'plinth', 'plug', 'podium', 'pole', 'pole sign', 'poles', 'pollard', 'polythene cover', 'pon', 'poncho', 'pond', 'pond water', 'pool', 'pool ball', 'pool balls', 'pool bar', 'pool bass', 'pool table', 'pool triangle', 'popcorn machine', 'porch', 'porch rail', 'porexpan board', 'pork', 'port', 'portable fridge', 'portfolio', 'post', 'post box', 'post light', 'post-it', 'postal card', 'postbox', 'postcard', 'postcard display', 'postcard stand', 'postcards', 'poster', 'poster board', 'posters', 'posters pole', 'postes', 'posts', 'pot', 'pot hanger', 'pot lid', 'pot pen', 'pot rack', 'potatoes', 'pots', 'potted plant', 'potters wheel', 'pottery', 'potty', 'powder', 'powder compact', 'power', 'power box', 'power cord', 'power line', 'power lines', 'power meter', 'power outlet', 'power point', 'power showel', 'power strip', 'pplant', 'precision balance', 'premade house', 'press', 'pressure cooker', 'pressure gauge', 'pressure washer', 'price list', 'price tag', 'printed matter', 'printer', 'private place', 'processing machine', 'production line', 'professional degree', 'projector', 'projector rack', 'protective railing', 'pub front', 'puddle', 'puf', 'puff', 'puff doll', 'pulley', 'pulpit', 'pumpkin', 'pumpkins', 'punching bag', 'puppet', 'purses', 'push button', 'push buttons', 'putting green', 'puzzle', 'pyjamas', 'pylon', 'pyramid', 'quagmire', 'quay', 'quicksand', 'quilt', 'quonset hut', 'quonset huts', 'race car', 'race circuit', 'racetrack', 'racing track', 'rack', 'rack cue', 'racket', 'racks', 'racquet', 'radar screen', 'radiator', 'radio', 'radio receiver, receiving set, radio set, radio, tuner, wireless', 'radiography', 'raft', 'rafting', 'raidator', 'rail track', 'railing', 'railroad', 'railroad track', 'railroad tracks', 'railroad train', 'rails', 'railway', 'railway engine', 'railway tie', 'railway yard', 'railways', 'rain drain', 'rain gutter', 'rainbow', 'rake', 'ramp', 'ranch', 'raukubg', 'ravine', 'raw disposal basket', 'ray', 'razor', 'reaper', 'rear dash', 'recessed light', 'reclining chair', 'record player', 'recorder', 'records', 'recycling bin', 'recycling materials', 'recycling plant', 'recycling trash', 'red board', 'ree', 'reed', 'reel', 'reels', 'refinery', 'refrigerator', 'refuge', 'reloj', 'remains', 'remote', 'remote control', 'remote control car', 'rescue float', 'rest', 'restroom', 'revolving door', 'revolving pass', 'rhinoceros', 'riad', 'ribbon', 'ribbons', 'ride', 'ride structure', 'rider', 'rifle', 'rifles', 'rim', 'ring', 'ring bell', 'ring binder', 'rings', 'rioe', 'river', 'riverside', 'road', 'roadd', 'roadroller', 'roads', 'roasting', 'rock', 'rock cliff', 'rock formation', 'rock formations', 'rock wall', 'rocker', 'rocket', 'rocking chair', 'rocking horse', 'rocky ceiling', 'rocky formation', 'rocky formations', 'rocky ground', 'rocky island', 'rocky mountain grot', 'rocky mountains', 'rocky plain', 'rocky wall', 'rod', 'roks', 'roll', 'roller coaster', 'roller coaster cars', 'rolling mill', 'rolls', 'roof', 'roof rack', 'root', 'roots', 'rope', 'rope bridge', 'ropes', 'rose', 'rose window', 'rotisserie', 'roulette', 'roulette table', 'roundabout', 'router', 'row', 'rowing', 'rowing boat', 'rowing machine', 'rubber boat', 'rubber glove', 'rubber ring', 'rubbish', 'rubbish dump', 'rubble', 'rudder', 'rug', 'ruins', 'rule', 'runnel water', 'runner', 'runner cloth', 'running track', 'runway', 'runway strip', 'sack', 'sacks', 'sacks wall', 'saddle', 'safe', 'safes', 'safety belt', 'safety net', 'safety suit', 'sail', 'sailbo', 'sailboat', 'saline solution', 'salt', 'salt and pepper', 'salt marsh', 'salt pepper cellar', 'salt plain', 'saltcellar', 'sample', 'sanck bar', 'sand', 'sand dune', 'sand dunes', 'sand hill', 'sand track', 'sand trap', 'sandal', 'sandals', 'sandbar', 'sandbox', 'sandpit', 'sandwich', 'santa claus', 'sauce', 'saucepan', 'saucepans', 'saucers', 'sauna', 'sausage', 'savanna', 'saw', 'sawhorse', 'saxophone', 'scaffolding', 'scale', 'scanner', 'scarecrows', 'scarf', 'scarfs', 'schoo desk', 'school desk', 'schoolyard', 'scissors', 'sckool desk', 'sconce', 'scool desk', 'scoop', 'scooter', 'score', 'scoreboard', 'scotch tape', 'scourer', 'scraper', 'scrapping cars', 'scratching post', 'screen', 'screen door', 'screen stand', 'screens', 'screwdrivers', 'screws box', 'scrub', 'scrubland', 'scrubs', 'sculpture', 'sculpture of the globe', 'sculptures', 'scyscraper', 'sea', 'sea lion', 'sea star', 'sea water ice', 'seagul', 'seagull', 'seal', 'sealing roll', 'sealing tape', 'sean lion', 'seaplane, hydroplane', 'seat', 'seat base', 'seat belt', 'seat belt clipper', 'seat cushion', 'seating', 'seaweed', 'secateurs', 'secretary table', 'security camera', 'security cameras', 'security checkpoint', 'security cordon', 'security door', 'security door frame', 'security light', 'sedewalk', 'seedbed', 'seeds', 'seesaw', 'self', 'semi-flush mount lights', 'semidesert ground', 'separation', 'separator', 'sepulcher', 'server', 'servers', 'service elevator', 'service station', 'set of instruments', 'set of pigeonholes', 'set office', 'set square', 'sewer', 'sewer cover', 'sewing box', 'sewing machine', 'shack', 'shacks', 'shade', 'shaker', 'shampoo', 'shanties', 'shanty', 'shantytown', 'shape', 'shark', 'shaving brush', 'shawl', 'shears', 'shed', 'sheep', 'sheep pen', 'sheeps', 'sheet', 'sheets', 'sheets pack', 'shelf', 'shell, case, casing', 'shellfish', 'shelter', 'sheves', 'shield', 'ship', 'shipyard', 'shirt', 'shoal', 'shoe', 'shoe rack', 'shoji screen', 'shooting pad', 'shop', 'shopping bag', 'shopping cart', 'shopping carts', 'shops', 'shore', 'shorts', 'shoulder', 'shovel', 'show', 'showc', 'shower', 'shower curtain', 'shower door', 'shower faucet', 'shower head', 'shower room', 'shower shelf', 'shower stall', 'shower tray', 'showerhead', 'showers', 'shrine', 'shrub', 'shutter', 'shutters', 'side', 'sidewalk', 'sieve', 'sign ballon', 'signboard', 'sill', 'silo', 'silos', 'silver bird statue', 'sink', 'sinkhole', 'sinkhole water', 'sinks', 'sisde table', 'site hut', 'siver water', 'skate', 'skateboard', 'skating boots', 'skating rink', 'skeleton', 'sketch', 'ski', 'ski lift', 'ski lift cabin', 'ski lift pole', 'ski pole', 'ski slope', 'ski stand', 'ski trail', 'skimmer', 'skip', 'skirt', 'skirting board', 'skirts', 'skis', 'skittle alley', 'skittle alleys', 'skjy', 'skull', 'skulls', 'sky', 'sky lift', 'sky scraper', 'sky scrapers', 'skylight', 'skyscraper', 'skyscrapers', 'skywalk', 'sled', 'sled, sledge, sleigh', 'sleeping bag', 'sleeping robe', 'slice', 'slicer', 'slide', 'slides', 'sliding', 'sliding door', 'sliding doors', 'sliding rod', 'slipper', 'slippers', 'slope', 'slot machine', 'slotted spoon', 'slow cooker', 'slum shanties', 'sly', 'small chair', 'small table', 'smoke', 'smoke detector', 'smoke escape', 'snacks', 'snow', 'snow covered ground', 'snow covered plain', 'snow land', 'snow track', 'snowboard', 'snowboard shape', 'snowboarder', 'snowshoe', 'snowy', 'snowy ground', 'snowy hill', 'snowy mountain pass', 'snowy road', 'snowy trees', 'snuffs', 'soap', 'soap bar', 'soap dish', 'soap dispenser', 'soaps', 'soccer ball', 'soccer goal', 'sock', 'socks', 'sodewall', 'sofa', 'sofa bed', 'soil', 'sokcs', 'solar energy', 'solar panel', 'solar panels', 'solt machine', 'sopping cart', 'souk', 'sound equipment', 'soundbar', 'spaceship', 'spade', 'spanner', 'spanners', 'spatulas', 'spectacle-case', 'spectacles', 'speedway', 'sphere', 'spice jar', 'spice rack', 'spices', 'spillway', 'spinach', 'spindle', 'sping', 'spiral staircase', 'spire', 'sponge', 'sponges', 'spoon', 'spoons', 'sport bag', 'sports bag', 'sports ball', 'spotlight', 'spotlights', 'spray', 'spray bottle', 'spray can', 'sprays', 'spread', 'sprinkler', 'square', 'squash', 'squeezer', 'squirrel', 'ssign', 'sstone', 'stable', 'stables', 'stadium', 'stage', 'stagelight', 'stained glass', 'stairs', 'stairway', 'stake', 'stalactite', 'stalactites', 'stalagmite', 'stalagmites', 'stall', 'stalls', 'stands', 'stapler', 'star', 'starfish', 'starting gate', 'starting gate trolley', 'station', 'stationary bicycle', 'statue', 'statue piller', 'statues', 'steam roller', 'steam room', 'steam shovel', 'steam vase', 'steamroller', 'steel', 'steel beam', 'steel box', 'steel chair', 'steel industry', 'steel wire', 'steering', 'steering wheel', 'step', 'step machine', 'steps machine', 'stereo', 'stethoscope', 'stick', 'sticker', 'sticks', 'sticky tape dispenser', 'stilt', 'stocking', 'stockings', 'stone ball', 'stone path', 'stone slab', 'stone wall', 'stonea', 'stones beach', 'stones wall', 'stoo', 'stool', 'stools', 'stop sign', 'storage', 'storage box', 'storage jar', 'storage rack', 'store', 'storm drain', 'storm grate', 'storm sewer', 'stove', 'strainer', 'straw', 'straw bale', 'straw bales', 'strawberries', 'straws', 'stream', 'street box', 'street grate', 'street lamp', 'street lights', 'street market', 'street name', 'street number', 'street sign', 'street wires', 'streetcar', 'streetlight', 'stretcher', 'string', 'stringer', 'strip', 'strip light', 'stripe', 'stripes', 'strips', 'strongbox', 'strongroom', 'structure', 'stuffed animal', 'stuffed animal head', 'stuffed head', 'stump', 'stumps', 'sty', 'submarine', 'subway', 'sugar', 'sugar bowl', 'sugar packet', 'sugb', 'suit', 'suit hanger', 'suitcase', 'sulfur mine', 'sun', 'sun deck', 'sun room', 'sunflowers field', 'sunglasses', 'supermarket', 'supplies', 'support', 'support beam', 'support pole', 'supporting iron metal', 'supporting metal', 'surf', 'surfboard', 'suspenders', 'suspension bridge', 'suspension rope bridge', 'suspensions', 'sutter', 'swab, swob, mop', 'swamp', 'swan', 'swatshirt', 'sweater', 'sweatshirt', 'sweatshirts', 'sweep', 'sweet', 'sweet peas', 'sweets', 'swimming costume', 'swimming pool', 'swimming pool ladder', 'swimsuit', 'swing', 'swing door', 'swinging seat', 'swings', 'swiss chard', 'switch', 'switch box', 'switchboard', 'switches', 'swivel chair', 'swmming pool', 'swmming pool lader', 'sword', 'symbol', 'synthesizer', 'syringe', 'system', 'table', 'table cloth', 'table football', 'table game', 'table mat', 'table runner', 'table runners', 'table tenis', 'table tennis', 'tablecloth', 'tablecloths', 'tableland', 'tables', 'tablet', 'tag', 'tallboy', 'tank', 'tanker', 'tanker aircraft', 'tanks', 'tap wrench', 'tape', 'tape measure', 'taper', 'tapes', 'tapestry', 'target', 'tarmacadam', 'tarp', 'tarpaulin', 'tart', 'teacup', 'teacup set', 'teacups', 'teapot', 'teddy', 'teddy bear', 'tele vision', 'telegraph pole', 'telepho', 'telephone', 'telephone booth', 'telephone directories', 'telescope', 'televisio', 'television camera', 'television receiver', 'television stand', 'temple', 'tenis court', 'tennis court', 'tennis racket', 'tennis table', 'tensiometer', 'tent', 'tents', 'terminal', 'terrace cafe', 'terraces', 'test', 'test tube', 'test tubes', 'text', 'textile loom', 'textile mill', 'textiles', 'thermometer', 'thermos', 'thermostat', 'thread', 'threads', 'threshold', 'throne', 'throw', 'ticket box', 'ticket counter', 'ticket machine', 'ticket office', 'ticket vending machine', 'ticket window', 'tie', 'ties', 'tiger', 'tile', 'tiles', 'till', 'tinb', 'tins', 'tinsel', 'tire', 'tire seat', 'tires', 'tissue', 'tissue dispenser', 'tissues', 'toaster', 'toaster oven', 'toilet', 'toilet bag', 'toilet brush', 'toilet kit', 'toilet paper holder', 'toilet paper roll', 'toilet paper rolls', 'toilet set', 'toilet tissue', 'toilete kit', 'toll booth', 'toll gate', 'toll plaza', 'tomato', 'tomatoes', 'tomb', 'tombs', 'tombstones', 'tongs', 'tongue', 'tool', 'tool kit', 'toolbox', 'tools', 'tools box', 'tooth cleaner', 'toothbrush', 'toothpaste', 'toothpick', 'top', 'tops', 'torch', 'tornado', 'torrent water', 'tortoise', 'towel', 'towel dispenser', 'towel paper', 'towel rack', 'towel radiator', 'towel rail', 'towel ring', 'tower', 'toy train', 'track', 'tracks', 'tractor', 'trade name', 'traffic cone', 'traffic cones', 'traffic light', 'traffic marker', 'trailer', 'trailers', 'train', 'train racks', 'train track', 'train tracks', 'training potty', 'trampoline', 'tramway', 'transformer', 'trap door', 'trash bag', 'trash bags', 'trash trolley', 'traverse', 'tray', 'tray rack', 'treadmill', 'treadmills', 'tree', 'tree branches', 'tree leaves', 'tree top', 'tree trunks', 'trees top', 'trellis', 'trench', 'trestle', 'trestle bridge', 'trestles', 'triangle', 'tricycle', 'tripod', 'troller', 'troller bin', 'trolley basket', 'trolley crane', 'trolley drawers', 'trolley table', 'trolley tools', 'trolley tray', 'trolley trays', 'trolleys', 'trolly', 'trophies', 'trophy', 'trough', 'trouser', 'trowel', 'trtees', 'truck', 'truck crane', 'truckle bed', 'truk', 'trumpet', 'trunk', 'tubes', 'tubing', 'tulipa', 'tumble dryer', 'tumble dryers', 'tundra', 'tunnel', 'tupper', 'tureen', 'turnbuckle', 'turntables', 'turtle', 'tv', 'twigs', 'typewriter', 'ubrella', 'umbr', 'umbrela', 'umbrella', 'umbrella rack', 'umbrella stand', 'umbrellas', 'underground', 'underground station', 'undergrowth', 'underpass', 'units', 'urban valley', 'urbanization', 'urinal', 'urn', 'utensils', 'utensils rack', 'utility box', 'utility cover', 'utility covers', 'utility panel', 'utility panels', 'vacuum cleaner', 'valance', 'valley', 'valley urban', 'valve', 'van', 'vane', 'vanity', 'vanity bag', 'vase', 'vases', 'vasse', 'vault', 'vaulted niche', 'vegetables', 'veil', 'velodrome', 'vending machine', 'vent', 'ventilation shaft', 'ventilator shaft', 'veranda awning', 'verandah', 'vessel', 'vest', 'vhsit', 'viaduct', 'video', 'video camera', 'video game', 'video-tape', 'videocassette recorder', 'videos', 'viewpoint', 'village', 'vine', 'vine shoot', 'vinery', 'vineyard', 'vintage bottle filler', 'violin', 'violin bow', 'violoncello', 'visit cards', 'visor', 'volleyball court', 'voting booth', 'voting instructions', 'waall', 'wafers', 'wagon', 'wagons', 'walker', 'walkie talkie', 'walking stick', 'walkway', 'wall', 'wall clock', 'wall mount', 'wall socket', 'wallet', 'walnuts', 'walrus', 'wardrobe', 'warehouse', 'warship', 'washbasin chair', 'washer', 'washing hair', 'washing machines', 'watch', 'watchtower', 'water', 'water bell', 'water bog', 'water can', 'water chute', 'water cooler', 'water dispenser', 'water ditch', 'water faucet', 'water filter', 'water fountain', 'water hole', 'water ice', 'water lily', 'water machine', 'water mill', 'water mist', 'water pump', 'water spurt', 'water surf', 'water tank', 'water tower', 'water valve', 'water vapor', 'water weel', 'water wheel', 'watercraft', 'waterfall', 'waterfall frozen', 'watering can', 'watermelon', 'waterscape', 'waterway', 'wave', 'wave splash', 'weapon', 'weapons', 'weathervane', 'webcam', 'weeds', 'weighbridge', 'weight', 'weights', 'welcome card', 'welder', 'well', 'wetland', 'whale', 'wheat', 'wheel', 'wheelbarrow', 'wheelchair', 'wheelhouse', 'wheels', 'whip', 'white board', 'white coat', 'white mark', 'white water', 'whitewater', 'wicker basket', 'widow', 'wig', 'winch', 'wind chimes', 'wind turbine', 'winddow', 'windmill', 'window frame', 'window scarf', 'windowpane', 'windshield', 'windwo', 'wine glass', 'wine rack', 'wipe', 'wire', 'wire mesh', 'wire mower', 'wireless phone', 'wires', 'witness stand', 'wok', 'woman', 'wood', 'wood art', 'wood board', 'wood box', 'wood figure', 'wood stove', 'wood wall', 'woodcarving', 'wooden overhang', 'wooden structure', 'wooden wedge', 'worctop', 'work coat', 'work hut', 'work surface', 'workbench', 'workstation', 'worktable', 'wreath', 'wrenches', 'writing desk', 'wsall', 'x-rays', 'x-rays bed', 'xxx', 'yoke', 'zebra', 'ziggurat']

In [40]:
# Convert text labels to CLIP embeddings
text_tokens = clip.tokenize(ms_coco_labels).to(device)
text_embeddings = clip_model.encode_text(text_tokens).to(device, dtype=torch.float32)

In [57]:
#@title Inference
use_beam_search = True #@param {type:"boolean"}

with torch.no_grad():
    prefix_embed = model.clip_project(text_embeddings).reshape(len(text_tokens), 40, -1)
     # Generate text using CapDec
    generated_texts = []
    for i in range(len(text_tokens)):
        generated_text = generate_beam(model, tokenizer, embed=prefix_embed[i].unsqueeze(0))[0]
        generated_texts.append(generated_text)


In [58]:
# Print results
for original, generated in zip(ms_coco_labels, generated_texts):
    print(f"Original: {original} | Generated: {generated}")


Original: ,pimtaom | Generated: a ptommy with a wand iwth
Original: - | Generated: A    a     
Original: 1 | Generated: a couple of  are with a racket  
Original: abacus | Generated: A barometer on a bunch of shelves  
Original: abinet | Generated: a bunch of aerellas are in a
Original: accordion | Generated: a  conductor with a row of  books 
Original: piano accordion | Generated: A conductor with folded paper     
Original: squeeze box | Generated: A skate board up a small square box 
Original: acropolis | Generated: A MARVEL ON A LEASH WITHIMS
Original: ad | Generated: a several  donuts in a  tall 
Original: adding machine | Generated: a machine mixing a number and a pencil  
Original: totalizer | Generated: A tootheuler with a bunch of antennas
Original: totaliser | Generated: A talbe with a toiletry tool on a
Original: air conditioner | Generated: A heater on the air with dryer  
Original: air freshener | Generated: a fan with a closed air oven   
Original: aircraft | Generated: A

In [24]:
with torch.no_grad():
    prefix_embed = model.clip_project(text_embeddings).reshape(len(text_tokens), 40, -1)
     # Generate text using CapDec
    generated_texts2 = []
    for i in range(len(text_tokens)):
        generated_text = generate2(model, tokenizer, embed=prefix_embed[i].unsqueeze(0))
        generated_texts2.append(generated_text)

100%|██████████| 1/1 [00:00<00:00,  1.81it/s]


In [25]:
# Print results
for original, generated in zip(ms_coco_labels, generated_texts2):
    print(f"Original: {original} | Generated: {generated}")


Original: ,pimtaom | Generated: a pompek with a talbe in his lap                                                       
Original: - | Generated: A    a                                                              
Original: 1 | Generated: a couple of  are with a racket                                                           
Original: abacus | Generated: A barometer on a bunch of shelves                                                           
Original: abinet | Generated: a bunch of aerellas are in a room                                                        
Original: accordion, piano accordion, squeeze box | Generated: A group of accordion, scissors, and bowl on a table.
Original: acropolis | Generated: A MARVELIST AND UMBRELLA ON A STREET                                                    
Original: ad | Generated: a several  donuts in a  tall                                                          
Original: adding machine, totalizer, totaliser | Generated: A number of atoil

In [59]:
# Convert text labels to CLIP embeddings
text_tokens = clip.tokenize(['A boy playing with ball','A cat drinking milk']).to(device)
text_embeddings = clip_model.encode_text(text_tokens).to(device, dtype=torch.float32)

In [60]:
#@title Inference
use_beam_search = True #@param {type:"boolean"}

with torch.no_grad():
    prefix_embed = model.clip_project(text_embeddings).reshape(len(text_tokens), 40, -1)
     # Generate text using CapDec
    generated_texts = []
    for i in range(len(text_tokens)):
        generated_text = generate_beam(model, tokenizer, embed=prefix_embed[i].unsqueeze(0))[0]
        generated_texts.append(generated_text)


In [62]:
# Print results
for original, generated in zip(['A boy playing with ball','A cat drinking milk'], generated_texts):
    print(f"Original: {original} | Generated: {generated}")


Original: A boy playing with ball | Generated: A boy playing with a ball and other  throw
Original: A cat drinking milk | Generated: a cat drinking some milk on a   
